# Tutorial 3: Insufficient Data (2D Range-only)

**API Version: Julia NavAbilitySDK.jl**

**Keywords:** Range-only, factor graph, under-determined, localization, mapping, SLAM, missing data, insufficient data, robotics, inference, Bayes tree, junction tree

## Overview

This example shows how non-Gaussian solutions occur from weak observability of desired variables from available data.  For this tutorial, we imagine a robot traveling around on a flat surface, in a rectangular trajectory making range measurements to beacons.  Let's assume WiFi power levels are used as a proxy for range measurements in this case.

![Example Overview](../../static/icra-3/example-wifi.png)

To illustrate the the underdetermined solution aspect, we simplify the problem in that WiFi ranging is assumed to produce a pure unimodal (Gaussian) measurement.  This tutorial will show how non-Gaussian behavior can arise because tha variables are weakly or underconstrained by the available measurement dimensions.  This tutorial is designed such that this occurs throughout the entire problem.

We build a factor graph in stages as the robot moves around the environment through pose/keyframe epochs.  The factor graph will be solved with the [open core Caesar.jl solver](https://github.com/JuliaRobotics/Caesar.jl) at each pose epoch, which produces the posterior marginal beliefs on each of the variables in the system.  After each pose epoch solution, we will look at the marginal belief estimates of all the variables in the system.

We assume the robot is traveling on a XY plane, starting at the origin along X and turning left along Y, then negative X, negative Y back to the origin.  Only four WiFi beacons are in the environment where the robot is moving.  Measurements to the WiFi beacons can only be included with there if the signal is within range.  Two of the beacon locations are known as prior information, while the other two beacons are at an unknown location and their location will be eastimated simultaneously to the robot pose location in the same factor graph system -- making this a simultaneous localization and mapping (SLAM) problem.

To further simplify the tutorial, the "odometry" measurement between consecutive poses  are also taken as range-only (i.e. distance-only) measurements.  These "odometry" factors provide less information than conventional wheel or visual odometry constraints might provide.

This tutorial shows [one of four mechanisms](https://juliarobotics.org/Caesar.jl/latest/concepts/why_nongaussian/) that can intoduce non-Gaussian behavior into a factor graph system, see other examples for other mechanisms.  Note that the techniques used in this tutorial can readily be combined with with methods from other tutorials.  For example, the pure Gaussian measurement ranging models used here can be replaced with ambiguous measurements shown in ICRA Tutorial 2.  Or, can be combined with uncertain data association (i,e. multi-hypothesis) measurement models for unknown beacon associations similat to the technique used in ICRA Tutorial 4.  Learn more from our peer-reviewed publications listed here: `CJLDocs/Literature`

### Signatures Used

`Point2`, `PriorPoint2`, `Point2Point2Range`, `MvNormal`, `Normal`.

## Loading The Data

The ground truth positions for vehicle positions GTp and landmark positions GTl can be loaded into memory directly with these values.  **Note,** we are using variable names
- `l1, l2` as ranging beacons with known locations,
- `l3, l4` as ranging beacons with initially unknown locations,  

These beacon landmarks must be in range before measurements can be made to them.  For the tutorial, we imagine a robot moving from one position to the next in the XY space between the landmarks.  We use ground truth positions to build the simulation, while the SLAM solution has to resolve estimates of the variables as the main exercise of the tutorial.  The robot positions are denoted as
- `x0, x1, ...`.

Ground truth data is as follows:

In [1]:
# Our dictionary of vehicle positions
GTp = Dict{Symbol, Vector{Float64}}()
GTp[:x0] = [0.0;0]
GTp[:x1] = [50.0;0]
GTp[:x2] = [100.0;0]
GTp[:x3] = [100.0;50.0]
GTp[:x4] = [100.0;100.0]
GTp[:x5] = [50.0;100.0]
GTp[:x6] = [0.0;100.0]
GTp[:x7] = [0.0;50.0]
GTp[:x8] = [0.0;-50.0]

# Our dictionary of landmark positions
GTl = Dict{Symbol, Vector{Float64}}()
GTl[:l1] = [10.0;30]
GTl[:l2] = [30.0;-30]
GTl[:l3] = [80.0;40]
GTl[:l4] = [120.0;-50];

## Loading Necessary Packages

An optional install line is kept here in case the packages are not yet installed in your environment for whatever reason.

In [ ]:
# optional install of packages, in case they are not available in your environment for whatever reason
import Pkg; Pkg.add("NavAbilitySDK");

Load the necessary packages,

In [8]:
using NavAbilitySDK
using LinearAlgebra

## Creating the Factor Graph

After loading the requried packages, lets start creating the factor graph using variables of type `Point2` (a.k.a. `Postion2`).

In [3]:
# you need a unique userId:robotId, and can keep using that across all tutorials
userId = "Guest"
robotId = "SDKjl_"*(string(uuid4())[1:4])

# also create a client connection
client = NavAbilityHttpsClient()

NavAbilityClient(NavAbilitySDK.var"#query#3"{Diana.Client}(Diana.Client(Diana.var"#Query#7"{Diana.var"#Query#3#8"}(Diana.var"#Query#3#8"(Core.Box(Dict{Any, Any}()), Core.Box("Bearer 0000"), Core.Box("https://api.d1.navability.io"))), Diana.var"#serverUrl#4"(Core.Box("https://api.d1.navability.io")), Diana.var"#setheaders#5"(Core.Box(Dict{Any, Any}())), Diana.var"#serverAuth#6"(Core.Box("Bearer 0000")))), NavAbilitySDK.var"#mutate#5"{Diana.Client}(Diana.Client(Diana.var"#Query#7"{Diana.var"#Query#3#8"}(Diana.var"#Query#3#8"(Core.Box(Dict{Any, Any}()), Core.Box("Bearer 0000"), Core.Box("https://api.d1.navability.io"))), Diana.var"#serverUrl#4"(Core.Box("https://api.d1.navability.io")), Diana.var"#setheaders#5"(Core.Box(Dict{Any, Any}())), Diana.var"#serverAuth#6"(Core.Box("Bearer 0000")))))

In [4]:
# You'll need a unique session number each time you run a new graph
sessionId = "Tutorial3_"*(string(uuid4())[1:4])
# context is the object to use below
context = Client(userId,robotId,sessionId)

Client: User=Guest, Robot=SDKjl_5dcd, Session=Tutorial3_575b

In [5]:
# Collect all the async responses and wait at the end
resultIds = Task[]

# first pose with no prior info about the initial numerical estimate
push!(resultIds, 
  addVariable(client, context, Variable("x0", :Point2))
);

# add three landmarks
for lbl in ["l1";"l2";"l3"]
  push!(resultIds, 
    addVariable(client, context, Variable(lbl, :Point2))
  );
end

The initial graph also has prior location information about each of the known beacons/landmarks `l1` and `l2`.  Let's go ahead and add those as factors:

In [6]:
# put prior on l1
f = Factor("l1f1", "PriorPoint2", ["l1"], 
  PriorPoint2Data(
    Z=FullNormal( GTl[:l1], diagm(ones(2)) )
  )
)
push!(resultIds, addFactor(client, context, f));

# put prior on l2
f = Factor("l2f1", "PriorPoint2", ["l2"], 
  PriorPoint2Data(
    Z=FullNormal( GTl[:l2], diagm(ones(2)) )
  )
)
push!(resultIds, addFactor(client, context, f));

# let's wait to make sure all the new additions are ready
waitForCompletion(client, resultIds; expectedStatuses=["Complete"])

The `PriorPoint2` is assumed to be a multivariate normal distribution of covariance `diagm(ones(2))`. Note the API `PriorPoint2(::SamplableBelief)` accepts any of the distribution objects that the Caesar.jl libraries support -- this is discussed further in subsection [Various `SamplableBelief` Distribution types](https://juliarobotics.org/Caesar.jl/latest/concepts/dataassociation/#Various-SamplableBelief-Distribution-Types).

## Adding Range Measurements Between Variables

Next we connect the three range measurements from the vehicle location `x0` to the three beacon landmarks `l1`, `l2`, and `l3`, respectively – and consider that the range measurements are completely relative between the vehicle and beacon position estimates:

In [11]:
# first range measurement from x0 to l1
rhoZ1 = norm(GTl[:l1]-GTp[:x0])
f = Factor("x0l1f1", "Point2Point2Range", ["x0";"l1"], 
  PriorPoint2Data(
    Z=Normal(rhoZ1, 2)
  )
)
push!(resultIds, addFactor(client, context, f));

# second range measurement from x0 to l2
rhoZ2 = norm(GTl[:l2]-GTp[:x0])
f = Factor("x0l2f1", "Point2Point2Range", ["x0";"l2"], 
  PriorPoint2Data(
    Z=Normal(rhoZ2, 2)
  )
)
push!(resultIds, addFactor(client, context, f));

# third range measurement from x0 to l3
rhoZ3 = norm(GTl[:l3]-GTp[:x0])
f = Factor("x0l3f1", "Point2Point2Range", ["x0";"l3"], 
  PriorPoint2Data(
    Z=Normal(rhoZ3, 2)
  )
)
push!(resultIds, addFactor(client, context, f));

The ranging measurement standard deviation of 2.0 or 3.0 is taken, assuming a Gaussian measurement assumption. Again, any distribution could have been used. The factor graph should look as follows:

In [12]:
println("https://app.navability.io/cloud/graph/?userId=$userId&robotStartsWith=$robotId&sessionStartsWith=$sessionId")

https://app.navability.io/cloud/graph/?userId=Guest&robotStartsWith=SDKjl_5dcd&sessionStartsWith=Tutorial3_575b
